In [1]:
from sqlalchemy import create_engine
from pathlib import Path
import pandas as pd
import helpers as h

@h.Timer()
def retrieve_data():
    """Downloads appropriate selection of data from PostgreSQL database"""

    with open('data/user.txt', 'r') as file:
        user = file.read().strip('\n')

    with open('data/pass.txt', 'r') as file:
        pw = file.read().strip('\n')

    with open('data/db_host.txt', 'r') as file:
        host = file.read().strip('\n')

    with open('data/db_port.txt', 'r') as file:
        port = file.read().strip('\n')

    with open('data/db_name.txt', 'r') as file:
        name = file.read().strip('\n')

    engine = create_engine(f'postgresql://{user}:{pw}@{host}:{port}/{name}')

    query = """SELECT * FROM blob WHERE "YEAR" = '2024' AND "MONTH" BETWEEN '03' AND '07' limit 100000;"""

    df = pd.read_sql(query, engine)

    parquet_path = Path('data/blob_data_100k.parquet')
    df.to_parquet(parquet_path, engine='pyarrow')

    print(f'Data saved to: {parquet_path}')

In [2]:
retrieve_data()

Data saved to: data/blob_data_100k.parquet
Function `retrieve_data` executed in 654.5031 sec, CPU: 2.50%, Memory: 1362.84MB


21946235 rows that match that criteria of date range.

In [4]:
# test count(*) of created dataset
import duckdb

parquet_path = Path('data/blob_data_100k.parquet')

check = duckdb.sql(
        f"SELECT count(*) FROM read_parquet('{parquet_path}')"
    ).to_df()

check.head()

,count_star()
0,100000


In [6]:
parquet_path = Path('data/blob_data_100k.parquet')

check = duckdb.sql(
        f"""SELECT count(*) FROM read_parquet('{parquet_path}') where "YEAR" = '2024' and "MONTH" = '03'"""
    ).to_df()

check.head()

,count_star()
0,0


In [12]:
parquet_path = Path('data/blob_data_100k.parquet')

df = duckdb.sql(f"""select * from read_parquet('{parquet_path}') limit 5;""").to_df()
df

,BLOB_ID,FULL_IMG_ID,POINT,SIZE,POLYGON_BOUNDRY_BOX,BLOB_POLYGON,GEO_HASHES,CONSTRUCTION_STAGE,BUILDING_TYPE,CS_MODEL_ID,...,IS_DUPLICATE,IS_BLOB_ON_IMAGE_EDGE,IS_VALID,IS_IMPUTED,IS_EXCLUDED,COUNTY,IS_OVERLAPPING_FOOTPRINT,IS_OVERLAPPING_ROAD,FOOTPRINT_ID,ROAD_DATA_ID
0,BID_a7258c95-7e44-424d-a227-1348f791ba39,FIID_58e9d2e6-11aa-4bc1-b1b0-951d312e72f5_2024...,POINT (-98.4602132804818 29.46763817191083),452,POLYGON ((-98.4602662037037 29.467706018518502...,POLYGON ((-98.4602662037037 29.467645833333318...,9v1zxkr,ROOF,NON-SF,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,False,True,False,False,BEXAR,True,False,FP_ID_1a055cc6-21f7-416a-90a3-f8afd3e13b1c,None
1,BID_b433a935-c506-4937-9be8-9b63b87ca696,FIID_58e9d2e6-11aa-4bc1-b1b0-951d312e72f5_2024...,POINT (-98.42449395672409 29.509311504753402),935,POLYGON ((-98.42457175925924 29.50939120370369...,POLYGON ((-98.42457175925924 29.50937268518517...,9v4pb7c|9v4pbk1,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,False,True,False,False,BEXAR,True,False,FP_ID_0e97e0c6-451c-4a83-a304-22d6a698cb19,None
2,BID_d9fa48dd-5ee7-4258-9d56-264d38f2eecc,FIID_87c905a3-ddd6-4a93-84cb-f07be70e1327_2024...,POINT (-96.87599626765075 32.86770283696455),4396,POLYGON ((-96.87620601851854 32.86782175925923...,POLYGON ((-96.87620601851854 32.86776157407404...,9vg4gx3|9vg4gx2,ROOF,NON-SF,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,True,False,True,False,False,DALLAS,True,False,FP_ID_41e0c77e-db4b-4702-bae9-17f823ef2cd8,None
3,BID_44517c93-b035-42b5-90dd-de2e341aa1d8,FIID_58e9d2e6-11aa-4bc1-b1b0-951d312e72f5_2024...,POINT (-98.42545971924973 29.455358060608745),623,POLYGON ((-98.4255162037037 29.455418981481465...,POLYGON ((-98.4255162037037 29.455418981481465...,9v4p860,ROOF,BACKGROUND,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,False,True,False,False,BEXAR,False,False,None,None
4,BID_cd928abd-adbf-4481-bc2f-627c7e0e27f9,FIID_58e9d2e6-11aa-4bc1-b1b0-951d312e72f5_2024...,POINT (-98.43521898353451 29.502309820768644),451,POLYGON ((-98.43526620370369 29.50235879629628...,POLYGON ((-98.43526620370369 29.50234953703702...,9v4pb49,ROOF,RESIDENTAL SF DETACHED,MID_6bf26a1d-f36e-4364-bc9d-d759d5c75d74,...,False,False,True,False,False,BEXAR,False,False,None,None


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   BLOB_ID                   5 non-null      object
 1   FULL_IMG_ID               5 non-null      object
 2   POINT                     5 non-null      object
 3   SIZE                      5 non-null      int64 
 4   POLYGON_BOUNDRY_BOX       5 non-null      object
 5   BLOB_POLYGON              5 non-null      object
 6   GEO_HASHES                5 non-null      object
 7   CONSTRUCTION_STAGE        5 non-null      object
 8   BUILDING_TYPE             5 non-null      object
 9   CS_MODEL_ID               5 non-null      object
 10  BT_MODEL_ID               5 non-null      object
 11  CITY                      5 non-null      object
 12  STATE                     5 non-null      object
 13  YEAR                      5 non-null      object
 14  MONTH                     5 no

In [14]:
type(df['YEAR'].iloc[0])

str

In [16]:
parquet_path = Path('data/blob_data_100k.parquet')

df = duckdb.sql(f"""select count(*) from read_parquet('{parquet_path}') where "YEAR" = '2024' and "MONTH" = '03';""").to_df()
df

,count_star()
0,0


In [17]:
duckdb.sql(f"DESCRIBE SELECT * FROM read_parquet('{parquet_path}')").show()


┌──────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│       column_name        │ column_type │  null   │   key   │ default │  extra  │
│         varchar          │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ BLOB_ID                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ FULL_IMG_ID              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ POINT                    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ SIZE                     │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ POLYGON_BOUNDRY_BOX      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ BLOB_POLYGON             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ GEO_HASHES               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ CONSTRUCTION_STAGE       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ BU

In [18]:
duckdb.sql(f"""SELECT DISTINCT "YEAR" FROM read_parquet('{parquet_path}');""").show()

┌─────────┐
│  YEAR   │
│ varchar │
├─────────┤
│ 2024    │
└─────────┘



In [19]:
duckdb.sql(f"""SELECT DISTINCT "MONTH" FROM read_parquet('{parquet_path}');""").show()

┌─────────┐
│  MONTH  │
│ varchar │
├─────────┤
│ 06      │
│ 07      │
└─────────┘



In [3]:
100000/21946235

0.004556590230624979

In [20]:
21946235/100000

219.46235

Create synthetic dataset to test our retrieve data function:

In [31]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
from pathlib import Path
import os
import json
import importlib
import helpers as h
import logging
import helpers as h

# Reload helpers if needed.
importlib.reload(h)

# Create a fake dataset with 10,000 rows.
num_rows = 10000
months = ['03', '04', '05', '06', '07']
data = {
    "BLOB_ID": [f"blob_{i}" for i in range(num_rows)],
    "YEAR": ['2024'] * num_rows,
    # Cycle through the months list.
    "MONTH": [months[i % len(months)] for i in range(num_rows)],
    "VALUE": [i for i in range(num_rows)],
    # Simulate CTID as an integer that increases sequentially.
    "ctid": [i for i in range(num_rows)]
}
fake_df = pd.DataFrame(data)

# Create a file-based SQLite database.
sqlite_db_path = "fake_blob.db"
engine = create_engine(f"sqlite:///{sqlite_db_path}")
# Save the fake DataFrame to a table named 'blob'.
fake_df.to_sql("blob", engine, if_exists="replace", index=False)

# Ensure the data directory exists.
os.makedirs("data", exist_ok=True)

# Remove any existing checkpoint file to start fresh.
checkpoint_path = Path("data/chunk_checkpoint.json")
if checkpoint_path.exists():
    checkpoint_path.unlink()

# Now run our retrieve_data function using the SQLite engine.
# In this test, we set chunk_size=2500 to produce 4 chunks.
h.retrieve_data(chunk_size=2500, engine_url=f"sqlite:///{sqlite_db_path}")

Function `retrieve_data` executed in 0.0282 sec, CPU: 22.90%, Memory: 1.92MB
